# Transformación de Datos

Transformación de datos es una de los procesos que más tiempo lleva en un un flujo de Data Science y, lamentablemente, muchas veces no es entretenido. Pero es importante no perder el objetivo de por qué lo hacemos, si bien en este momento de la carrera puede parecer un poco abstracto. Por un lado, los modelos de Machine Learning que usemos, que van a "aprender" de nuestros datos, sólo entienden de números. Esto quiere decir que, si en un dataset como el de Titanic hay una columna de género con dos valores, *male* y *female*, tendremos que llevarlo a una representación que los modelos entiendan. En este caso, es sencillo, asignamos un "0" y un "1" a cada uno de los valores y problema resuelto. En el fondo, hay una hipótesis implícita: el género nos sirve para predecir supervivencia, por lo que queremos mantener esta información y hacerla comprensible para nuestros modelos. Eso es un atributo (feature): un pedazo de información potencialmente bueno para aprender de nuestros datos o hacer predicciones útiles.

## Transformación de Datos con Pandas

Vamos a mostrar algunas funcionalidades de Transformación de Datos con Pandas con algunos ejemplos sobre el dataset de Titanic.

Primero importamos las librerías y cargamos el dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

In [2]:
df = pd.read_csv('DS_Clase_05_titanic.csv')
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Variables Numéricas

En general, las variables numéricas podrían ser usadas sin demasiado preprocesamiento, ya que ya están en un formato que los modelos entienden. Sin embargo, muchas veces no es ese el caso. Algunas de las cosas que se suelen hacer son:
* Discretización y binning
* Reescalar (más adelante en el curso)
* Combinar con otras variables (más adelante en el curso)


**Discretización y binning con Pandas**

Si ya sabemos qué bines usar, Pandas es muy útil.

#### `cut()`

In [5]:
bins = [0,3,12,18,60,100]

In [6]:
cats = pd.cut(df.Age, bins)
cats.head(10)

0    (18, 60]
1    (18, 60]
2    (18, 60]
3    (18, 60]
4    (18, 60]
5         NaN
6    (18, 60]
7      (0, 3]
8    (18, 60]
9    (12, 18]
Name: Age, dtype: category
Categories (5, interval[int64]): [(0, 3] < (3, 12] < (12, 18] < (18, 60] < (60, 100]]

In [5]:
df['Age_bins'] = cats
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_bins
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,"(18, 60]"
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,"(18, 60]"
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,"(18, 60]"
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,"(18, 60]"
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,"(18, 60]"


In [9]:
bins = [0,3,12,18,60,100]
labels = ['bebe', 'niño', 'adolescente', 'adulto', 'anciano']

In [10]:
cats = pd.cut(df.Age, bins, labels = labels)
cats.head(10)

0         adulto
1         adulto
2         adulto
3         adulto
4         adulto
5            NaN
6         adulto
7           bebe
8         adulto
9    adolescente
Name: Age, dtype: category
Categories (5, object): [bebe < niño < adolescente < adulto < anciano]

In [11]:
df['Age_labels'] = cats
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_labels
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,adulto
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,adulto
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,adulto
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,adulto
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,adulto


### Nominales

Este es uno de los tipos de *encoding* más comunes que vamos a tener que hacer. Empecemos con un ejemplo, el género en el Titanic. Ya dijimos que queremos llevar los valores `male` y `female` a `0` y `1`. El orden no importa, lo único importante es que no nos olvidemos cuál es cuál. Esto, en Pandas, lo podemos hacer con 

#### `map()`

En primer lugar, definimos un diccionario, que es donde tendremos el *mapeo*.

In [12]:
diccionario = {'male': 0, 'female': 1}

Luego, usamos la función para crear una columna nueva o reemplazar la columna ya existente con los nuevos valores.

In [14]:
df['Sex_Map'] = df.Sex.map(diccionario)

In [11]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_bins,Age_labels,Sex_Map
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,"(18.0, 60.0]",adulto,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,"(18.0, 60.0]",adulto,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,"(18.0, 60.0]",adulto,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,"(18.0, 60.0]",adulto,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,"(18.0, 60.0]",adulto,0
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,NaN,NaN,0
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,"(18.0, 60.0]",adulto,0
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,"(0.0, 3.0]",bebe,0
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,"(18.0, 60.0]",adulto,1
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,"(12.0, 18.0]",adolescente,1


### Variables Dummies

Una variable dummy toma como valor 0 ó 1 para indicar la presencia o ausencia de algún atributo categórico. Es decir, prácticamente igual a lo que acabamos de hacer en el paso anterior. La función `get_dummies()` hace automáticamente esto en un dataframe sobre las columnas que nosotros le indiquemos.

Estudiar qué hace la función en la celda siguiente. ¿Cuál es una diferencia apreciable con `map()`?¿Qué ocurre si no le indicamos sobre cuáles columnas obtener las variables dummies? Probarlo e interpretar.

In [25]:
pd.get_dummies(df['Sex'])
# pd.get_dummies(df[['Sex','Age']])
# pd.get_dummies(df)

pd.get_dummies(list('ABCDA'))

,A,B,C,D
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0


Y, para agregar al Dataframe original, podemos hacer así:

In [13]:
df = pd.concat([df, pd.get_dummies(df['Sex'])], axis=1)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_bins,Age_labels,Sex_Map,female,male
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,"(18, 60]",adulto,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,"(18, 60]",adulto,1,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,"(18, 60]",adulto,1,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,"(18, 60]",adulto,1,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,"(18, 60]",adulto,0,0,1


### Ejercitación

Vamos a trabajar con el dataset 'DS_Clase_10_Heart.csv'. Dejamos una breve descripción de qué representan algunas columnas.

    -slope_of_peak_exercise_st_segment: the slope of the peak exercise ST segment, an electrocardiography read out indicating quality of blood flow to the heart
    -thal: results of thallium stress test measuring blood flow to the heart
    -resting_blood_pressure: resting blood pressure
    -chest_pain_type: chest pain type
    -num_major_vessels: number of major vessels colored by flourosopy
    -fasting_blood_sugar_gt_120_mg_per_dl: fasting blood sugar > 120 mg/dl
    -resting_ekg_results: resting electrocardiographic results
    -serum_cholesterol_mg_per_dl: serum cholestoral in mg/dl
    -oldpeak_eq_st_depression: oldpeak = ST depression induced by exercise relative to rest, a measure of abnormality in electrocardiograms
    -max_heart_rate_achieved: maximum heart rate achieved (beats per minute)
    -exercise_induced_angina: exercise-induced chest pain (0: False, 1: True)

**Muy importante:** para responder las consignas, no es necesario que programen *desde cero*. De hecho, es raro programar *desde cero*, sino que todo el tiempo reciclamos código de otros trabajos. Por lo tanto, para responder las preguntas, recuerden dónde hicieron algo similar, copien las celdas pertinentes y adáptenlas a este problema. No nos vamos a cansar de recordarles esto.

1. Hacer un análisis exploratorio de datos y responder las siguientes preguntas: 
    1. ¿Qué tipo de dato hay en cada columna desde el punto de vista de la programación (entero, float, etc.)?¿Qué tipo de dato son según lo visto hoy en la clase (nominal, ordinal, numérico, etc.)?¿Hay alguna relación entre ambos mundo?¿Qué podemos hacer si no sabemos una variable de qué tipo es?
    1. ¿Hay alguna columna que puedan descartar para este análisis?
    1. ¿Cuántos (y cuáles donde consideren apropiado) son los valores únicos de cada columna?
    1. ¿Cómo están correlacionadas las variables? Recuerden descartar las columnas que no tengan números. Interpretar.
    
2. Usar las herramientas vistas hoy en clase para transformar los datos de las columnas que consideren importante transformar. Una vez hecho eso, vuelvan a hacer el cuadro de correlaciones. ¿Hay información nueva?

**Nota**: A medida que vayan respondiendo estas preguntas, ir agregando celdas *markdown* intercaladas con el código donde expliquen brevemente las conclusiones a las que llegaron.

**Algunas pistas**:
* Para abrir el dataset y ver qué tipos de datos hay en cada columna, y cuántos valores únicos tienen, algunas funcionalidades de Pandas pueden ser útiles.
* Para obtener una visión rápida del dataset, hay una función de Seaborn que hace precisamente eso.
* Para obtener una linda tabla de correlaciones, en el Notebook de la Clase 7 hay un ejemplo que pueden adaptar.

In [29]:
df = pd.read_csv('DS_Clase_10_Heart.csv')
df.head() 
#buscar pandas profile

,patient_id,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina,heart_disease_present
0,0z64un,1,normal,128,2,0,0,2,308,0.0,male,45,170,0,0
1,ryoo3j,2,normal,110,3,0,0,0,214,1.6,female,54,158,0,0
2,yt1s1x,1,normal,125,4,3,0,2,304,0.0,male,77,162,1,1
3,l2xjde,1,reversible_defect,152,4,0,0,0,223,0.0,male,40,181,0,1
4,oyt4ek,3,reversible_defect,178,1,0,0,2,270,4.2,male,59,145,0,0


In [31]:
df.describe()

,slope_of_peak_exercise_st_segment,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,age,max_heart_rate_achieved,exercise_induced_angina,heart_disease_present
count,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000
mean,1.550000,131.311111,3.155556,0.694444,0.161111,1.050000,249.211111,1.010000,54.811111,149.483333,0.316667,0.444444
std,0.618838,17.010443,0.938454,0.969347,0.368659,0.998742,52.717969,1.121357,9.334737,22.063513,0.466474,0.498290
min,1.000000,94.000000,1.000000,0.000000,0.000000,0.000000,126.000000,0.000000,29.000000,96.000000,0.000000,0.000000
25%,1.000000,120.000000,3.000000,0.000000,0.000000,0.000000,213.750000,0.000000,48.000000,132.000000,0.000000,0.000000
50%,1.000000,130.000000,3.000000,0.000000,0.000000,2.000000,245.500000,0.800000,55.000000,152.000000,0.000000,0.000000
75%,2.000000,140.000000,4.000000,1.000000,0.000000,2.000000,281.250000,1.600000,62.000000,166.250000,1.000000,1.000000
max,3.000000,180.000000,4.000000,3.000000,1.000000,2.000000,564.000000,6.200000,77.000000,202.000000,1.000000,1.000000


In [32]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 15 columns):
patient_id                              180 non-null object
slope_of_peak_exercise_st_segment       180 non-null int64
thal                                    180 non-null object
resting_blood_pressure                  180 non-null int64
chest_pain_type                         180 non-null int64
num_major_vessels                       180 non-null int64
fasting_blood_sugar_gt_120_mg_per_dl    180 non-null int64
resting_ekg_results                     180 non-null int64
serum_cholesterol_mg_per_dl             180 non-null int64
oldpeak_eq_st_depression                180 non-null float64
sex                                     180 non-null object
age                                     180 non-null int64
max_heart_rate_achieved                 180 non-null int64
exercise_induced_angina                 180 non-null int64
heart_disease_present                   180 non-null int64
dtype